<a href="https://colab.research.google.com/github/karino2/US-patent-analysis/blob/triplet_loss_colab/colab/bert_extract_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT feature extraction for grants and app**

Extract bert feature for triplet loss.

Porting to colab of feature extraction part below.

https://github.com/yoheikikuta/US-patent-analysis/blob/master/notebook/tripletloss_model_with_circulum_learning.ipynb

In [0]:
import os
import datetime
import pickle
import gzip

import tensorflow as tf

In [2]:
tf.__version__

'1.13.1'

In [3]:
!git clone -b docker https://github.com/yoheikikuta/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 220.79 KiB | 2.94 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [4]:
!ls

bert  sample_data


In [0]:
!mkdir ./bert/data

In [0]:
from google.colab import auth
auth.authenticate_user()

In [7]:
!gsutil cp gs://karino2-uspatent/features/test_grants_ids.pkl.gz ./bert/data/
!gsutil cp gs://karino2-uspatent/features/training_app_ids.pkl.gz ./bert/data/

Copying gs://karino2-uspatent/features/test_grants_ids.pkl.gz...
/ [1 files][  3.0 MiB/  3.0 MiB]                                                
Operation completed over 1 objects/3.0 MiB.                                      
Copying gs://karino2-uspatent/features/training_app_ids.pkl.gz...
/ [1 files][794.7 KiB/794.7 KiB]                                                
Operation completed over 1 objects/794.7 KiB.                                    


In [8]:
!ls ./bert/data

citations_info_2000.df.gz  test_grants_ids.pkl.gz  training_app_1000.df.gz
grants_for_2000.df.gz	   testset_app_1000.df.gz  training_app_ids.pkl.gz


In [0]:
with gzip.open("./bert/data/test_grants_ids.pkl.gz", 'rb') as f:
     (test_ids, grants_ids) = pickle.load(f)
with gzip.open("./bert/data/training_app_ids.pkl.gz", 'rb') as f:
     training_ids = pickle.load(f)

**Check tpu name**

In [5]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.81.75.66:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 10635577033367471859),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 976606133020435086),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4501356155637751213),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13632525968536816513),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8641085653688703972),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 369903914460703330),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9085190748269255527),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 10709987216607880277),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 4476668466533816

### Bert setup

In [0]:
VOCAB_CONFIG_PATH=' gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12' #@param {type:"string"}

In [13]:
!gsutil cp {VOCAB_CONFIG_PATH}/vocab.txt ./bert/model/patent/vocab.txt
!gsutil cp {VOCAB_CONFIG_PATH}/bert_config.json ./bert/model/patent/bert_config.json

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/vocab.txt...
/ [1 files][226.1 KiB/226.1 KiB]                                                
Operation completed over 1 objects/226.1 KiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json...
/ [1 files][  313.0 B/  313.0 B]                                                
Operation completed over 1 objects/313.0 B.                                      


In [14]:
!ls ./bert/model/patent

bert_config.json  vocab.txt


In [0]:
import sys
sys.path.append("./bert")

In [0]:
import tokenization

In [0]:
tokenizer = tokenization.FullTokenizer(
    vocab_file="./bert/model/patent/vocab.txt", do_lower_case=True)

In [0]:
CLS_ID = tokenizer.vocab["[CLS]"]
SEP_ID = tokenizer.vocab["[SEP]"]

In [0]:
max_seq_length = 512

In [0]:
def convert_single_sentence(ids_a_input, uniqueue_id):
  ids_a = ids_a_input
  # Account for [CLS] and [SEP] with "- 2"
  if len(ids_a_input) > max_seq_length - 2:
    ids_a = list(ids_a_input)[0:(max_seq_length - 2)]  
      

  # The convention in BERT is:
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  input_ids = []
  segment_ids = []
  input_ids.append(CLS_ID)
  segment_ids.append(0)
  for token in ids_a:
    input_ids.append(token)
    segment_ids.append(0)
  input_ids.append(SEP_ID)
  segment_ids.append(0)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  return (input_ids, input_mask, segment_ids, uniqueue_id)


In [0]:
BASE_TRAINING=10000
BASE_TEST = 20000
BASE_GRANTS = 30000


# unique_id, tokens, input_ids, input_mask, input_type_ids
def feature_tuplist_to_feature_dict(ftups):
    dic = {}
    dic['input_ids'] = [tup[0] for tup in ftups]
    dic["input_mask"] = [tup[1] for tup in ftups]
    dic["input_type_ids"] = [tup[2] for tup in ftups]
    dic['unique_ids'] = [tup[3] for tup in ftups]
    return dic

In [0]:
import os

import modeling
import tokenization

from extract_features  import model_fn_builder


In [0]:
bert_config = modeling.BertConfig.from_json_file("./bert/model/patent/bert_config.json")

In [15]:
BUCKET = 'karino2-uspatent' #@param {type:"string"}
OUTPUT_DIR = 'gs://{}/mlstudy-phys/bert/models/patent_1000'.format(BUCKET)

INIT_CHECK_POINT_FILE='model.ckpt-1602' #@param {type:"string"}
INIT_CHECK_POINT = '{}/{}'.format(OUTPUT_DIR, INIT_CHECK_POINT_FILE)

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
print('***** Init checkpoint: {} *****'.format(INIT_CHECK_POINT))

***** Model output directory: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000 *****
***** Init checkpoint: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000/model.ckpt-1602 *****


In [0]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.vocab_file = "./bert/model/patent/vocab.txt"
        self.use_tpu = True
        self.output_dir = OUTPUT_DIR
        # self.data_dir = "./bert/data/1000"
        self.init_checkpoint = INIT_CHECK_POINT
        self.predict_batch_size = 16
        # TPU related
        self.num_tpu_cores = 8
        self.tpu_name = TPU_ADDRESS
        
        # following parameters are not used anymore. (because we create feature by hand)
        self.do_lower_case = True
        self.max_seq_length = 512
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        # TPU related. Some of these value have positive int not to make TPUEstimator angry (even though these are not used...).
        self.eval_batch_size = 32
        self.train_batch_size = 32 
        self.iterations_per_loop = 1000


In [0]:
FLAGS = FLAGS()

In [0]:
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [0]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS.init_checkpoint,
    use_tpu=FLAGS.use_tpu,
    layer_indexes=[-1],
    use_one_hot_embeddings=FLAGS.use_tpu)

In [0]:
# https://github.com/kyzhouhzau/BERT-NER/issues/19

def model_fn_wrapper(features, labels, mode, params):
  tspec = model_fn(features, labels, mode, params)
  pred_dict = {'predictions': tspec.predictions['layer_output_0']}
  return tf.contrib.tpu.TPUEstimatorSpec(
          mode=tspec.mode, predictions=pred_dict, scaffold_fn=tspec.scaffold_fn)


In [0]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    # model_fn=model_fn,
    model_fn=model_fn_wrapper,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

### Feature extraction

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
training_app_input_fdict = feature_tuplist_to_feature_dict([convert_single_sentence(one_app, BASE_TRAINING+idx) for idx, one_app in enumerate(training_ids)])

In [0]:
import numpy as np
import pandas as pd

In [0]:
def ids_to_input_fdict(ids, base_id):
  return feature_tuplist_to_feature_dict([convert_single_sentence(one_app, base_id+idx) for idx, one_app in enumerate(ids)])

def fdict_to_features(fdict):

  def _batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(fdict)    
    return test_ds.batch(params['batch_size'])

  feature_results =  np.empty((0,768), float)
  print(datetime.datetime.today())
  result = estimator.predict(_batch_input_function, yield_single_examples=True)
  
  for one in result:
    feature_results = np.append(feature_results, one['predictions'][0].reshape(1,768), axis=0 )  
  print(datetime.datetime.today())
  return feature_results


In [58]:
train_features = fdict_to_features(training_app_input_fdict)

2019-03-01 03:45:56.035574
2019-03-01 03:46:21.520941


In [59]:
len(train_features)

1000

In [60]:
train_features.shape

(1000, 768)

In [61]:
train_features[0, 0:5]

array([-1.04389739,  0.88131171, -0.35072595,  0.79031169, -0.26134047])

In [78]:
%%time
with gzip.open("training_app_1000_features.pkl.gz", 'w') as f:
     pickle.dump(train_features, f)

CPU times: user 8.26 s, sys: 7.6 ms, total: 8.27 s
Wall time: 8.28 s


In [79]:
!gsutil cp training_app_1000_features.pkl.gz gs://karino2-uspatent/features/training_app_1000_features.pkl.gz

Copying file://training_app_1000_features.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/3.1 MiB.                                      


In [80]:
!ls -l training_app_1000_features.pkl.gz

-rw-r--r-- 1 root root 3262353 Mar  1 04:19 training_app_1000_features.pkl.gz


### Create test set application feature

In [0]:
test_app_input_fdict = ids_to_input_fdict(test_ids, BASE_TEST)

In [87]:
test_app_features = fdict_to_features(test_app_input_fdict)

2019-03-01 04:24:11.729741
2019-03-01 04:24:41.610773


In [88]:
test_app_features[0, 0:5]

array([-0.2944594 ,  0.57456857, -0.08040974,  1.03909576,  0.08170709])

In [89]:
test_app_features.shape

(1000, 768)

In [90]:
%%time
with gzip.open("test_app_1000_features.pkl.gz", 'w') as f:
     pickle.dump(test_app_features, f)

CPU times: user 8.15 s, sys: 2.93 ms, total: 8.15 s
Wall time: 8.16 s


In [91]:
!gsutil cp test_app_1000_features.pkl.gz gs://karino2-uspatent/features/test_app_1000_features.pkl.gz

Copying file://test_app_1000_features.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  3.1 MiB/  3.1 MiB]                                                
Operation completed over 1 objects/3.1 MiB.                                      


### Create grants2000 feature

In [0]:
grants_input_fdict = ids_to_input_fdict(grants_ids, BASE_GRANTS)

In [93]:
grants_features = fdict_to_features(grants_input_fdict)

2019-03-01 04:30:05.723136
2019-03-01 04:30:46.824327


In [94]:
grants_features.shape

(2524, 768)

In [95]:
grants_features[0, 0:5]

array([-0.73496956,  1.23776925, -0.64043194,  0.28553247, -0.10149363])

In [97]:
%%time
with gzip.open("grants_2000_features.pkl.gz", 'w') as f:
     pickle.dump(grants_features, f)

CPU times: user 20.9 s, sys: 3.18 ms, total: 20.9 s
Wall time: 20.9 s


In [98]:
!gsutil cp grants_2000_features.pkl.gz gs://karino2-uspatent/features/grants_2000_features.pkl.gz

Copying file://grants_2000_features.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  7.9 MiB/  7.9 MiB]                                                
Operation completed over 1 objects/7.9 MiB.                                      


### Check to kikuta-san's data

Seems different. We don't know the reason.

In [81]:
!mkdir kikuta_feature

mkdir: cannot create directory ‘kikuta_feature’: File exists


In [64]:
!gsutil cp gs://yohei-kikuta/mlstudy-phys/bert/data/1000/* kikuta_feature/

Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/bert_extracted_feature_test_1000.pkl...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/bert_extracted_feature_train_1000.pkl...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/citations_info_2000.df.gz...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/dev.tsv...
| [4 files][122.2 MiB/122.2 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/grants_feature_2000.pkl...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/grants_for_2000.df.gz...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/testset_app_1000.df.gz...
Copying gs://yohei-kikuta/mlstudy-phys/bert/data/1000/testset_app_feature_1000.pkl...
Copy

In [65]:
!ls -l kikuta_feature/training_app_feature_1000.pkl

-rw-r--r-- 1 root root 6187667 Mar  1 04:03 kikuta_feature/training_app_feature_1000.pkl


In [66]:
ls -l training_app_ids.pkl.gz

-rw-r--r-- 1 root root 813752 Mar  1 02:14 training_app_ids.pkl.gz


In [0]:
kikuta_training_df = pd.read_pickle("kikuta_feature/training_app_feature_1000.pkl")

In [73]:
kikuta_training_df['feature'][0]  == train_features[0,]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [74]:
kikuta_training_df['feature'][0][0:5]

array([-0.80122775,  0.36823475, -0.40176541,  0.12356944,  0.34096408])

In [75]:
train_features[0, 0:5]

array([-1.04389739,  0.88131171, -0.35072595,  0.79031169, -0.26134047])

In [76]:
kikuta_training_df['app_id'][0]

14222691